# Part 2 - Warehouse Analytics

Building on the simulated product-level data generated in the [previous notebook](01_stock_simulation.ipynb), this section aggregates across all product IDs to provide a daily view of warehouse operations. This allows us to evaluate staffing needs, delivery patterns and operational costs at an operational level.

In this section, we will examine for each day:

- Total number of items entering the warehouse.
- Total number of items leaving the warehouse.
- The total inventory levels across all product IDs.
- The total number of orders that went unfulfiled.
- Total inbound and outbound shipments.
- Truck and van utilization.
- Workers needed to meet demand.
- Simulated human errors. 

We will consider these metrics for both replenishment strategies, weekly scheduled and JIT deliveries, to allow for a more robust comparison.


## Set-up

Import the required modules and the previous dataset:

In [1]:
import pandas as pd
import numpy as np
import math

np.random.seed(16)

products_df = pd.read_csv("../data/warehouse_products.csv")

Select only the relevant columns:

In [2]:
selected_columns = ["date",
                    "warehouse_id",
                    "inbound_units_weekly",
                    "actual_outbound_weekly",
                    "inventory_level_weekly",
                    "unmet_demand_weekly",
                    "inbound_units_jit",
                    "actual_outbound_jit",
                    "inventory_level_jit",
                    "unmet_demand_jit"]

df = products_df[selected_columns]

Aggregate the columns by grouping by both date and warehouse ID:

In [3]:
group_df = df.groupby(["date", "warehouse_id"])
group_df = group_df.agg({
    "inbound_units_weekly": "sum",
    "actual_outbound_weekly": "sum",
    "inventory_level_weekly": "sum",
    "unmet_demand_weekly": "sum",
    "inbound_units_jit": "sum",
    "actual_outbound_jit": "sum",
    "inventory_level_jit": "sum",
    "unmet_demand_jit": "sum"})
group_df = group_df.reset_index()

Rename the aggregated columns to more appropriate names:

In [4]:
group_df = group_df.rename(columns={
    "actual_outbound_weekly": "orders_fulfilled_weekly",
    "unmet_demand_weekly": "missed_sales_weekly",
    "actual_outbound_jit": "orders_fulfilled_jit",
    "unmet_demand_jit": "missed_sales_jit"
})

## Warehouse Operations

### Outbound Shipments

Every day

In [5]:
van_capacity = 100
group_df["outbound_shipments_weekly"] = group_df["orders_fulfilled_weekly"] / van_capacity
group_df["outbound_shipments_jit"] = group_df["orders_fulfilled_jit"] / van_capacity

In [6]:
group_df["outbound_shipments_weekly"] = group_df["outbound_shipments_weekly"].apply(np.ceil).astype("int")
group_df["outbound_shipments_jit"] = group_df["outbound_shipments_jit"].apply(np.ceil).astype("int")

### Inbound Shipments

In [8]:
large_truck_capacity = 1000

group_df["inbound_shipments_weekly"] = group_df["inbound_units_weekly"] / large_truck_capacity
group_df["inbound_shipments_weekly"] = group_df["inbound_shipments_weekly"].apply(np.ceil).astype("int")
group_df["truck_utilization_weekly"] = group_df["inbound_units_weekly"] / (group_df["inbound_shipments_weekly"] * large_truck_capacity)

In [9]:
small_truck_capacity = 500

group_df["inbound_shipments_jit"] = group_df["inbound_units_jit"] / small_truck_capacity
group_df["inbound_shipments_jit"] = group_df["inbound_shipments_jit"].apply(np.ceil).astype("int")
group_df["truck_utilization_jit"] = group_df["inbound_units_jit"] / (group_df["inbound_shipments_jit"] * small_truck_capacity)

### Warehouse Utilization

In [7]:
warehouse_capacity = 2500

group_df["utilization_weekly"] = group_df["inventory_level_weekly"] / warehouse_capacity
group_df["utilization_jit"] = group_df["inventory_level_jit"] / warehouse_capacity

## Staffing and Human Factors

### Staff Count

In [10]:
def determine_staff_count(row, outbound_column):
    outbound_orders = row[outbound_column]
    workers_needed = math.ceil(outbound_orders / 50)
    return workers_needed

group_df["staff_count_weekly"] = group_df.apply(determine_staff_count, args=("orders_fulfilled_weekly",), axis=1)
group_df["staff_count_jit"] = group_df.apply(determine_staff_count, args=("orders_fulfilled_jit",), axis=1)

### Human Error

In [11]:
group_df.to_csv("../data/warehouse_daily.csv", index=False)